In [1]:
import os

os.environ['TIKA_LOG_PATH'] = os.getcwd()

In [2]:
from glob import glob
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'

In [3]:
pdfs = glob('*/*.pdf')
len(pdfs)

2012

In [4]:
pdfs

['pdf/pub_20130918_P.U. (B) 375-PENGECUALIAN DARIPADA PERENGGAN 5(1) (a) DAN (b).pdf',
 'pdf/P.U. (A) 109 2007.pdf',
 'pdf/pua_20120511_P.U. (A) 133.pdf',
 'pdf/pua_20150709_P.U. (A) 143 - Perintah Kawalan Bekalan (Barang-Barang Kawalan) (NO 3) 2015.pdf',
 'pdf/pua_20121218_P.U. (A) 450-Perintah Cukai Aktiviti Perniagaan Labuan (Pengecualian) 2012.pdf',
 'pdf/PUB 564.pdf',
 'pdf/Act 92 -P.U. (A) 357 of 1980.pdf',
 'pdf/PUA 21.pdf',
 'pdf/pua_20130514_P.U. (A) 154.pdf',
 'pdf/pub_20121227_P.U. (B) 425_ My Creative warta.pdf',
 'pdf/PUB 528 (2022).pdf',
 'pdf/pua_20150727_P.U. (A) 169_2015.pdf',
 'pdf/pua_20120719_P.U. (A) 220-Potongan Premium Insurans Pengimport (Pembatalan).pdf',
 'pdf/pua_20120511_P.U. (A) 132-Transfer Pricing Rules BM  BI (11 5 2012)(FINAL E-WARTA).pdf',
 'pdf/PUB 355.pdf',
 'pdf/pua_20120424_Peraturan2 Pilihan Raya (Pendaftaran Pemilih)(Pindaan)_P.U. (A) 106.pdf',
 'pdf/pua_20121001_P.U. (A) 321.pdf',
 'pdf/pua_20131017_P.U. (A) 313-PerintahKastam(MinyakMentahPetrol

In [5]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()

In [6]:
def parse_tika(filename):
    raw_xml = parser.from_file(filename, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            t.append(t_)
        except Exception as e:
            pass
    return t

In [7]:
!rm -rf json
!mkdir json

In [8]:
from tqdm import tqdm
import json

def loop(pdfs):
    pdfs, _ = pdfs
    for f in tqdm(pdfs):
        filename = os.path.join('json', f.replace('/', '-') + '.json')
        if os.path.exists(filename):
            continue

        try:
            b = parse_tika(f)
            with open(filename, 'w') as fopen:
                json.dump(b, fopen)
        except Exception as e:
            print(e)

In [9]:
loop((pdfs[:2],0))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 57.83it/s]


In [11]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [12]:
import mp

In [ ]:
mp.multiprocessing(pdfs, loop, cores = 8, returned = False)

 67%|███████████████████████████▍             | 168/251 [00:11<00:06, 12.60it/s]

In [ ]:
!du -hs json